# Cube Packcage
The custom cube package is preinstalled.  just load the library and start to use it 
<hr/>


### Load Library and Login to CubeAPI

In [21]:
# load all library, it is already installed
library(devtools)
library(httr)
library(jsonlite)
library(stringr)
library(rapportools)
library(htmlwidgets)
library(rlist)
library(data.table)
library(cloudml)
library(readr)
library(logger)
library(cube)
setwd("~/")
log_threshold(INFO)

In [22]:
# create a CubeAPI object
cube_api = CubeAPI$new()

INFO [2021-01-05 20:42:31] Version: 0.0.2
INFO [2021-01-05 20:42:31] http://10.105.16.22/metadata-service/


In [ ]:
# only need to login once per month
# after run, click on "verification_uri_complete" to finish verification
cube_api$login()

### Get Metadata Definition

In [23]:
# retrieve metadata element
response = cube_api$get_element()

INFO [2021-01-05 20:42:34] GET: http://10.105.16.22/metadata-service/metadata_definition/element/
INFO [2021-01-05 20:42:37] status_code: 200


In [ ]:
str(response)

In [24]:
# get the content of the response
content = content(response)

In [ ]:
str(content)

In [25]:
# to data frame
data.frame(do.call(rbind, content$results))[c("id", "element_label", "element_collection_label", "element_abbreviation")]

id,element_label,element_collection_label,element_abbreviation
<list>,<list>,<list>,<list>
138,Animal,Animals,AA
85,Assay Type,Assay Types,AS
125,Biobank,Biobank,BB
137,Data Set,Data Sets,DS
115,Disease,Diseases,DS
112,Drug Treatment,Drug Treatments,DR
122,File Set,File Sets,FS
118,Funding Source,Funding Sources,FD
117,Human Donors,Human Donors,HU


### Get Bucket data from metadata collection

In [26]:
storage_info = cube_api$get_metadata_collection_storage_info()

INFO [2021-01-05 20:42:49] GET: http://10.105.16.22/metadata-service/metadata_repository/collection/
INFO [2021-01-05 20:42:50] status_code: 200
INFO [2021-01-05 20:42:50] number of accession_ids: 9
INFO [2021-01-05 20:42:50] GET: http://10.105.16.22/metadata-service/metadata_repository/element_instance/
INFO [2021-01-05 20:42:52] status_code: 200
INFO [2021-01-05 20:42:52] number of row returned: 4


In [27]:
storage_info

,accession_id,uri,bucket_name,file_name
,<chr>,<chr>,<chr>,<chr>
1,JAXDS00004,gs://jax-cube-prd-ctrl-01-project-data/study_4-1/4.1_NMR_to_201016.txt?authuser=1,gs://jax-cube-prd-ctrl-01-project-data/study_4-1,4.1_NMR_to_201016.txt
2,JAXDS00003,https://storage.cloud.google.com/jax-cube-prd-ctrl-01-project-data/study_4-1/4.1_GTT_to_201016.txt?authuser=1,gs://jax-cube-prd-ctrl-01-project-data/study_4-1,4.1_GTT_to_201016.txt
3,JAXDS00002,https://storage.cloud.google.com/jax-cube-prd-ctrl-01-project-data/study_4-1/4.1_DEXA_to_201016.txt?authuser=1,gs://jax-cube-prd-ctrl-01-project-data/study_4-1,4.1_DEXA_to_201016.txt
4,JAXDS00001,https://storage.cloud.google.com/jax-cube-prd-ctrl-01-project-data/study_4-1/4.1_BODY_WEIGHT_to_201016.txt,gs://jax-cube-prd-ctrl-01-project-data/study_4-1,4.1_BODY_WEIGHT_to_201016.txt


In [ ]:
## get bucket data
bucket_data = get_bucket_data(bucket_name = storage_info[2, 3], file_name = storage_info[2, 4] )
bucket_data

<br/><br/>

### Get Metadata Definition with a Parameter

In [ ]:
response = cube_api$get_element(element_id = 85)

In [ ]:
str(content(response))

### Get Metadata with accession ids 

In [ ]:
response = cube_api$get_element_instance(element_id = 85, accession_ids = c("JAXAS00031", "JAXAS00032", "JAXAS00034"))

In [ ]:
str(content(response)$results)

### Get Metadata Collection

In [ ]:
response = cube_api$get_metadata_collection()

In [ ]:
json = response_json_to_data(response)

In [ ]:
str(json$results)

In [ ]:
accession_ids = json$results[,2][[1]]$accession_id
accession_ids

In [ ]:
response = cube_api$get_element_instance(accession_ids = c(accession_ids))

In [ ]:
results = content(response)$results
results[[4]][[4]]

In [ ]:
data_url = results[[4]][[4]][[1]][[4]]
url_parts = str_split(data_url, "/")
bucket_name = paste0("gs://", url_parts[[1]][4], "/", url_parts[[1]][5])
file_name =  url_parts[[1]][6]

# load the data from the GCP bucket
data_dir = gs_data_dir( bucket_name )

# read the file into a dataframe
df <- read.table(file.path(data_dir, file_name), sep = '\t',header = TRUE)
df

### Call Any Get endpoint
Call any get end point.  All the endpoints are listed here: https://metadata-service-sqa.azurewebsites.net/swagger/

In [ ]:
# specify a end point
response = cube_api$get_end_point("metadata_definition/property/?element_id=85")

In [ ]:
content(response)$count

In [ ]:
# another end point
response = cube_api$get_end_point("/metadata_repository/collection_item/1/")

In [ ]:
content(response)

### Post endpoint

In [ ]:
response = cube_api$post_element_instance_filter(element_id = 122, page = 1, page_size = 2)

In [ ]:
str(content(response))

### More Information

In [ ]:
?CubeAPI

In [ ]:
?Cube